In [1]:
import os
from sys import argv, stdout, stderr
from ROOT import TCanvas, TLegend, gROOT, gStyle, TFile, TIter, gSystem, TH1, TDirectory
import sys
gROOT.SetStyle("Plain")
gROOT.SetBatch(True)

Welcome to JupyROOT 6.10/09


In [2]:
def positivize(histogram):
    output = histogram.Clone()
    for i in range(output.GetSize()):
        if output.GetArray()[i] < 0:
            output.AddAt(0, i)
    return output

In [4]:
filelist = ['DYFIT.root', 'SMHFIT.root', 'TTFIT.root', 'STFIT.root', 'EWKFIT.root', 'DibosonFIT.root', 'SignalFIT.root']
name = ['DY', 'SMH', 'TTbar', 'SingleTop', 'EWK', 'Diboson', 'LFV125']
f = TFile("CollMassFit.root", "UPDATE")
sjet = {"0Jet", "1Jet", "2Jet", "2JetVBF"}
for s in sjet:
    d = f.mkdir(s)
    for i, fil in enumerate(filelist):
        fi = TFile(fil)
        hall = fi.Get('TightOS'+s+'/bdtDiscriminator')#m_t_CollinearMass
        htfakes = fi.Get('TauLooseOS'+s+'/bdtDiscriminator')
        hmfakes = fi.Get('MuonLooseOS'+s+'/bdtDiscriminator')
        hmtfakes = fi.Get('MuonLooseTauLooseOS'+s+'/bdtDiscriminator')
        hmt = htfakes.Clone()
        hmt.Add(hmfakes)
        hfakes = hmt.Clone()
        hfakes.Add(hmtfakes, -1)
        hfakes = positivize(hfakes)
        h = hall.Clone()
        h.Add(hfakes, -1)
        h = positivize(h)
        h.SetName(name[i])
        h.SetDirectory(d)
        f.Write()   

In [5]:
sjet = {"0Jet", "1Jet", "2Jet", "2JetVBF"}
for s in sjet:
    fi = TFile("dataFIT.root")
    f = TFile("CollMassFit.root", "UPDATE")
    d = f.GetDirectory(s)
    h = fi.Get('TightOS'+s+'/bdtDiscriminator')
    h.SetName("data_obs")
    h.SetDirectory(d)
    f.Write()

In [6]:
sjet = {"0Jet", "1Jet", "2Jet", "2JetVBF"}
for s in sjet:
    fi = TFile("dataFIT.root")
    f = TFile("CollMassFit.root", "UPDATE")
    d = f.GetDirectory(s)
    htfakes = fi.Get('TauLooseOS'+s+'/bdtDiscriminator')
    hmfakes = fi.Get('MuonLooseOS'+s+'/bdtDiscriminator')
    hmtfakes = fi.Get('MuonLooseTauLooseOS'+s+'/bdtDiscriminator')
    hmt = htfakes.Clone()
    hmt.Add(hmfakes)
    h = hmt.Clone()
    h.Add(hmtfakes, -1)
    h = positivize(h)
    h.SetName("Fakes")
    h.SetDirectory(d)
    f.Write()